## Usage
This project provides the three ways to update CPI data, such as append load, trancate and load, and incremental load.
- "Get the Latest Data" enables users to get the latest CPI data by inputing any dates.
- "Load Data" shows the three ways to load the data up to the date which users input.
- "Compare Performance" illustrates the difference of processing speed among these three ways.

These services will make users decide which way to choose for their development.

## General Preparation
- Import necessary libraries.
- Define function of loading CPI data into dataframe.
- Define function of getting the latest data.

In [18]:
import pandas as pd
import duckdb
from load_functions import load_data, get_latest_data

## Get the Latest Data
Write any pull date from 1998-11-01 to 2025-2-28 and run the code to see the result of the latest five data.

This project manually vefities that the following validation works.
- If users select the date before 1998-11-01, pull date is set to be 1998-11-01.
- If users select the date after 2025-2-28, pull date is set to be 2025-2-28.


In [19]:
import datetime

url = 'https://www.philadelphiafed.org/-/media/FRBP/Assets/Surveys-And-Data/real-time-data/data-files/xlsx/pcpiMvMd.xlsx?sc_lang=en&hash=E41A743DC6423F950B10C3DE7A4F674D'
pull_date = '2024-12-01'

pull_date = datetime.datetime.strptime(pull_date, '%Y-%m-%d').date()

min_date = datetime.date(1998, 11, 1)
max_date = datetime.date(2025, 2, 28)

pull_date = min(max(pull_date, min_date), max_date)

latest_data = get_latest_data(load_data(url), str(pull_date))
latest_data.tail()


,DATE,CPI
932,2024:09,314.686
933,2024:10,315.454
934,2024:11,316.441
935,2024:12,NaN
936,2025:01,NaN


## Load Data

- Users initially have to set a pull date.
- Runnign each way of loading date shows the latest five data which updated by each way.

In [20]:
user_pull_date = '1998-12-01'

1. Append Load

In [21]:
from load_functions import update_append

con_append = duckdb.connect('cpi_append.duckdb')
con_append.execute('DROP TABLE IF EXISTS cpi_table')
con_append.execute("""
    CREATE TABLE cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_append(user_pull_date, url, con_append)
updated_records = con_append.execute('SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5').fetchdf()
print(updated_records)

con_append.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


2. Truncate and Load

In [22]:
from load_functions import update_truncate

con_truncate = duckdb.connect('cpi_truncate.duckdb')
con_truncate.execute('DROP TABLE IF EXISTS cpi_table')
con_truncate.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_truncate(user_pull_date, url, con_truncate)
updated_records = con_truncate.execute("SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5").fetchdf()
print(updated_records)

con_truncate.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


3. Incremental Load

In [23]:
from load_functions import update_incremental

con_incremental = duckdb.connect('cpi_incremental.duckdb')
con_incremental.execute('DROP TABLE IF EXISTS cpi_table')
con_incremental.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_incremental(user_pull_date, url, con_incremental)
updated_records = con_incremental.execute("SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5").fetchdf()
print(updated_records)

con_incremental.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


## Compare Performance

- Users initially have to set start and end date of loading data.
- Runnign each way of loading date shows processing times by each way.

In [30]:
from load_functions import measure_time

start_date = '2024-12-01'
end_date = '2025-02-01'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

1. Check the performance of Append Load.

In [31]:
con_append = duckdb.connect('cpi_append.duckdb')
con_append.execute('DROP TABLE IF EXISTS cpi_table')
con_append.execute("""
    CREATE TABLE cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f'Total Execution Time of Append: {measure_time(update_append, date_range, url, con_append):.2f} seconds')

con_append.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


Total Execution Time of Append: 211.37 seconds


c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


2. Check the performance of Truncate and Load.

In [32]:
con_truncate = duckdb.connect('cpi_truncate.duckdb')
con_truncate.execute('DROP TABLE IF EXISTS cpi_table')
con_truncate.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f'Total Execution Time of Truncate: {measure_time(update_truncate, date_range, url, con_truncate):.2f} seconds')

con_truncate.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not t

Total Execution Time of Truncate: 389.45 seconds


3. Check the performance of Incremental Load.

In [33]:
con_incremental = duckdb.connect('cpi_incremental.duckdb')
con_incremental.execute('DROP TABLE IF EXISTS cpi_table')
con_incremental.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f'Total Execution Time of Incremental: {measure_time(update_incremental, date_range, url, con_incremental):.2f} seconds')

con_truncate.close()

c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  updated_data.to_sql('cpi_table', con, if_exists='append', index=False)
c:\Users\oiwah\OneDrive\デスクトップ\2025_Spring_Python\lab4\arshiya-hiroyuki-api\load_functions.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not 

Total Execution Time of Incremental: 354.76 seconds


## Conclusion

The processing times were shortest for append load (211.37 seconds), followed by incremental load (354.76 seconds), and longest for truncate and load (389.45 seconds). In terms of speed alone, append load is the fastest since it simply adds new data, while truncate and load is the slowest as it reuploads the entire dataset each time. Incremental load falls in between, as it updates only the differences.These results imply that processing times are proportional to the amount of data uploaded.

From a data quality perspective, append load risks degrading accuracy because historical data is periodically updated, leading to inconsistencies. However, replacing the entire dataset every time is unnecessary—updating only the changes is sufficient. Balancing both data quality and processing speed, the most appropriate approach for handling CPI data is incremental load, as it efficiently maintains accuracy while keeping processing time manageable.